# CS224n: Natural Language Processing with Deep Learning

<img src="https://drive.google.com/uc?export=view&id=1I0h5HOcIt-Ga-4Y1ZfSlM09s2e6fQagn">
Trick: 
1. copy the web address: < img src="https://drive.google.com/uc?export=view&id=XXX">
2. go to the google drive and find the file you want to insert ([details](https://stackoverflow.com/questions/15557392/how-do-i-display-images-from-google-drive-on-a-website))
3. replace XXX in step 1 with the ID of the file

## Chapter 1 Introduction and Word Vectors

### 1.1 Tasks in NLP 

NPL tackles several tasks at varying level of difficulty:
- Easy
 - Spell Checking
 - Keyword Search
 - Finding Synonyms
- Medium
 - Parsing information from websites, documents, etc.
- Hard 
 - Machine Translation (e.g. Translate Chinese text to English)
 - Semantic (语义的，语义学的) Analysis (What is the meaning of query statement?)
 - Coreference (互参，互见)(e.g. What does "he" or "it" refer to given a docu- ment?)
 - Question Answering (e.g. Answering Jeopardy questions).

### 1.2 How to represent words?

To perform well on most NLP tasks we first need to have some notion of similarity and difference between words. With **word vectors**, we can quite easily encode this ability in the vectors themselves (using distance measures such as Jaccard, Cosine, Eu- clidean, etc).

### 2 Word Vectors

Let’s dive into our first word vector and arguably the most simple, the __one-hot vector__: Represent every word as an R|V|×1 vector with all 0s and one 1 at the index of that word in the sorted english language. In this notation, |*V*| is the size of our vocabulary

<font size=3>\begin{equation}
w^{aardvark}=\begin{bmatrix}
    1       \\
    0    \\
    0 \\
    \vdots \\
    0
\end{bmatrix}, 
w^{a}=\begin{bmatrix}
    0       \\
    1    \\
    0 \\
    \vdots \\
    0
\end{bmatrix}, 
w^{at}=\begin{bmatrix}
    0       \\
    0    \\
    1 \\
    \vdots \\
    0
\end{bmatrix}, \dots
w^{zebra}=\begin{bmatrix}
    0       \\
    0    \\
    0 \\
    \vdots \\
    1
\end{bmatrix}
\end{equation}
    </font>

We represent each word as a completely independent entity.
<font size=4>$$(w^{hotel})^{T}w^{motel}=(w^{hotel})^{T}w^{cat}$$</font>

However, this repsentation of words doesn't give any information on the relationships between words. So we need to reduce the size of this space from $R|V|$ to something smaller to find the relationships. 

### 3 SVD Based Method. 

Another method used is the Singular Value Decompositon based method. There are several steps that we need to take to find the right representation of the words.

1. Loop over a massive dataset and accumulate word co-occurrence counts in some form of a matrix X

2. Perform Singular Value Decomposition on X to get a USVT decomposition.
3. Use the rows of U as the word embeddings for all words in our dictionary. 

####3.1 Word-Document Matrix 

First we make bold conjuecture that words that are related will often appear in the same documents. We use this fact to build a word-document matrix, X in the following manner: Loop over billions of documents and for each time word $i$ appears in document $j$, we add one to entry $X_{ij}$. 

But this matrix could be huge ( ${\rm I\!R}^{V \times M}$ where $M$ is the number of the document), and it scales with the number of documents $M$. We need to find a better way to fix this problem. 

An example application can be found [HERE Topic 4. Linear Classification and Regression](https://colab.research.google.com/drive/1Dk-OUFB7yhj_yMGkNYPbtL3h0ZqXVfF2).

####3.2 Window based Co-occurrence Matrix
Similar logic in **3.1** applies here, however, <font color='color'>we count the number of times each word appears inside a window of a particular size around the word of interest</font>. For example, consider the corpus that contains only three sentences and window size be 1: 
1. I enjoy flying.
2. I like NLP.
3. I like deep learning.

The resulting counts matrix will then be:

<img src="https://drive.google.com/uc?export=view&id=1wv2kq2qOpCH7bpX3NCV9VI3QzW08fOsj">



###3.3 Applying SVD to the cooccurrence matrix
Next we need to perform SVD on X. This step involves ranking the importance/revelance  of the words based on their singular/eignevalues. The higher the eignevlaues, the more revelant or important is that word. Altervaitively, we use singular value to be a standard for the words selection and reduce the size of the vocalbulary we have. 

*<font size=3, color='grey'>But now the question is why do we use **<font color='blue'>Eigenvalue/Singular Value</font>** to represent the importance of the word? and How? </font>*

___
I will explain this using *Page Rank Algorithm* by google to answer this problem. 

Suppose for instance, that we have a small Internet consisting of just 4 web sites www.page1.com, www.page2.com, www.page3.com, www.page4.com, referencing each other in the manner suggested by the picture:

![page rank](http://pi.math.cornell.edu/~mec/Winter2009/RalucaRemus/Lecture3/Images/webpages.jpg)

We "translate" the picture into a directed graph with 4 nodes, one for each web site. When web site $i$ references $j$, we add a directed arrow between node $i$ and node  $j$ in the graph. For the purpose of computing their page rank, we ignore any navigational links such as back, next buttons, as we *only care about the connections between different web sites*. For instance, Page1 links to all of the other pages, so node 1 in the graph will have outgoing edges to all of the other nodes. Page3 has only one link, to Page 1, therefore node 3 will have one outgoing edge to node 1.

<img src="http://pi.math.cornell.edu/~mec/Winter2009/RalucaRemus/Lecture3/Images/graf1.PNG" align="center"/>


In our model, each page should transfer evenly its importance to the pages that it links to. Node 1 has 3 outgoing arrows, so it will pass on  of its importance to each of the other 3 nodes. Node 3 has only one outgoing arrow, so it will pass on all of its importance to node 1.


Let us denote by A the transition matrix of the graph, 

\begin{equation}
A=\begin{bmatrix}
    0 & \color{skyblue}{0} & \color{red}{1} &\color{green}{ \frac{1}{2} }      \\
    \frac{1}{3} & \color{skyblue}{0} & \color{red}{0} &\color{green}{ 0 }       \\
  \frac{1}{3} & \color{skyblue}{\frac{1}{2}} & \color{red}{0} &\color{green}{ \frac{1}{2} }    \\
   \frac{1}{3} & \color{skyblue}{\frac{1}{2}} &  \color{red}{0} &\color{green}{ 0 }  
\end{bmatrix}
\end{equation}

Let us denote by $x1,\, x2,\, x3,\,$, and $x4$ the importance of the four pages. Analyzing the situation at each node we get the system:


\begin{cases}
               x_{1}=1 \cdot x_{3}+\frac{1}{2} \cdot x_{4} \\
                x_{2}=\frac{1}{2} \cdot x_{1}\\
                x_{3}=\frac{1}{3} \cdot x_{1}+\frac{1}{2} \cdot x_{2}+\frac{1}{2} \cdot x_{4}\\
                 x_{4}=\frac{1}{3} \cdot x_{1}+\frac{1}{2} \cdot x_{2}
\end{cases}

This is equivalent to asking for the solutions of the equations 
\begin{equation}
A \cdot \begin{bmatrix}
    x_{1}      \\
     x_{2}    \\
     x_{3} \\
     x_{4}
\end{bmatrix}=\lambda \cdot \begin{bmatrix}
    x_{1}      \\
     x_{2}    \\
     x_{3} \\
     x_{4}
\end{bmatrix}\end{equation}

Then what is $\lambda$, that is the **<font color='blue'>Eigenvalue/Singular Value</font>** we know that there are many eigenvalues related to this $A$ matrix: $\lambda_{1}\approx 1$, $\lambda_{2}\approx -0.27875 $.... However, we need the eigenvector/PageRank vectore to be positive so that it can be undertand in a probailistic way. So we will only keep the  eigenvalues that give positive eigenvectors and that is $\lambda_{1}\approx 1$. Becasue when $\lambda_{2}\approx -0.27875 $([How to calculate eigenvalues and eigenvectors?](https://www.scss.tcd.ie/~dahyotr/CS1BA1/SolutionEigen.pdf)), the corresponding eigenvector becomes $\begin{bmatrix}1.05362 \\-1.25992 \\ -0.79370 \\ 1\end{bmatrix}$.
So the genvectors corresponding to the eigenvalue 1 are of the form $c \cdot \begin{bmatrix}12 \\ 4 \\ 9 \\ 6\end{bmatrix}$. We can normalize this eigenvctor into a probabilistic eigenvector corresponding to the eigenvalue 1 with the sum of all entries equal to 1. That is $\frac{1}{31} \cdot \begin{bmatrix}12 \\ 4 \\ 9 \\ 6\end{bmatrix} \approx  \begin{bmatrix}0.38 \\ 0.12 \\ 0.29 \\ 0.19\end{bmatrix}$.  That is we could have equations such as $x_{1}=1/3 x_{2}$ or similar equations, that just attribute the weights of importances to each of the node **<font color='red'>. This final PageRank Factor reflect only the relative importance of the nodes</font>**([Reference](http://pi.math.cornell.edu/~mec/Winter2009/RalucaRemus/Lecture3/lecture3.html)).
___


So now the answer is very clear, the eigenvalue will be the index of the relative importance of the words. And we can pick the most important words based on this in the SVD method.  For example,  we have colors being represented with 8-bits as follows. But, now we want to compress this into 4-bits only. Which bits should we retain, lower or higher ones?

___

![bits](https://cdn-images-1.medium.com/max/800/1*pH-qJHdEM6yF2Wm4rCTjQg.png)

___
When the bits are reduced, most important information is to identify the color not the shades of different colors. Hence, lower bits are the most important ones. So our task in SVD is actually capture these lower bits. 

###3.3 Applying SVD to the cooccurrence matrix

The singular value decomposition will break this into best rank approximation capturing information from most relevant to least relevant ones.

**SVD**. the singular-value decomposition of an $m\times n$ real or complex matrix $\mathbf {X}$  is a factorization of the form$\mathbf {USV^{T}}$, where $\mathbf {U}$  is an $m\times m$ real or complex unitary matrix, $\mathbf{S}$ is an $m\times n$ rectangular diagonal matrix with **non-negative real numbers** on the diagonal, and $\mathbf {V}$  is an $n\times n$ real or complex unitary matrix. The diagonal entries $\mathbf{\sigma_{i}}$ of $\mathbf{S}$ are known as the **singular values** of $\mathbf {X}$.  The columns of $\mathbf {U}$  and the columns of $\mathbf {V}$  are called the left-singular vectors and right-singular vectors of $\mathbf {X}$ , respectively.

The singular-value decomposition can be computed using the following observations:

- The left-singular vectors of $\mathbf {X}$ are a set of orthonormal eigenvectors of $\mathbf {XX^{T}}$.
- The right-singular vectors of $\mathbf {X}$ are a set of orthonormal eigenvectors of $\mathbf {X^{T}X}$.
- The non-zero singular values of $\mathbf {X}$ (found on the diagonal entries of $\mathbf {S}$) are the square roots of the non-zero eigenvalues of both $\mathbf {X^{T}X}$ and $\mathbf {XX^{T}}$.

So basically SVD is a sandwich with two eigevectors and one eigenvalue of $\mathbf {XX^{T}}$ & $\mathbf {XX^{T}}$([Reference](https://towardsdatascience.com/art-of-vector-representation-of-words-5e85c59fee5)).

After we perform SVD, the cosine similarity will be visable between different word using the eigenvalues and eigenvector of $\mathbf{X}$. 
___
![SVD](https://cdn-images-1.medium.com/max/800/1*dfTpIl9pK9pyYQOpq9pJxg.png)

<font size='1.5' color='grey'>From product of [XXt] matrix a low rank matrix capturing important features is constructed. the latent co-occurrence between {system, machine} and {human, user} has become visible. See, the **red** and **blue** parts in given figure.</font>
___
Besides, we will break this original word matrix into best rank approximation capturing information from most relevant to least relevant ones. 
___
![SVD1](https://cdn-images-1.medium.com/max/800/1*iZMtpjZCwISUObwV4IDC0Q.png)

<font size='1.5' color='grey'>SVD theorem tells us that $u_{1}$ ,$v_{1}$ and $\sigma_{1}$ store the most important information in X. Subsequent terms stores less and less important information.</font>
___

Now we cut the singular values off at some index k based on the desired percentage variance captured:
$$\frac{\sum_{i=1}^{k}\sigma_{i}}{\sum_{i=1}^{|V|}\sigma_{i}}$$
___
**Applying SVD to** X

<img src="https://drive.google.com/uc?export=view&id=1yp19Ju_Vs_MD49b1zu_zo0n18vIyCpcJ">

**Reducing dimensionality by selecting first k singular vectors**:
<img src="https://drive.google.com/uc?export=view&id=1JxWLzLSTHw7yQjhGW-weRDq-oVXrRE-L">
___

Both of these methods give us word vectors that are more than sufficient to encode semantic and syntactic (part of speech) information but are associated with many other problems:

- The dimensions of the matrix change very often (new words are added very frequently and corpus changes in size). 
 - SVD based methods do not scale well for big matrices and it is hard to incorporate new words or documents. Computational cost for a m × n matrix is $O(mn^2)$
- The matrix is extremely sparse since most words do not co-occur.
- The matrix is very high dimensional in general ($\approx 10^6 \times 10^6$)
- Quadratic cost to train (i.e. to perform SVD)
- Requires the incorporation of some hacks (砍，劈) on X to account for the drastic (jiliede激烈)imbalance in word frequency.


Some solutions to exist to resolve some of the issues discussed above: efficient use of the statistics
- Ignore function words such as "the", "he", "has", etc.
- Apply a ramp window – i.e. weight the co-occurrence count based on distance between the words in the document.
- Use Pearson correlation and set negative counts to 0 instead of using just raw count.

###4 Iteration Based Methods - Word2vec

#### 4.1  Word2vec Theory
This new method called iteration based method solves many of these issues in the previous methods in a far more elegant manner. Word2vec (Mikolov et al. 2013) is a framework for learning word vectors. 

The basic idea of this model is as follows:

- We need to establish a large corpus of text
- Each word in a fixed vocabulary is represented by a <font color='red'>vector</font>
- Go through each position t in the text, which has a <font color='red'>center word c</font> and <font color='red'>context (“outside”) words o</font>
- Use the <font color='red'>similarity of the word vectors</font> for c and o to calculate
<font color='red'>the probability</font> of o given c (or vice versa)
- <font color='red'>Keep adjusting the word vectors</font> to maximize this probability

Graphically, the possibility of of words given its center word can be illustarted as follows: 
___
<img src="https://drive.google.com/uc?export=view&id=13apLip8M9Jt7zWPDwZFzyT3b9JXjWr-8" width="800"/>
___
We first calculate the $P(turning\,|\, into)$, $P(banking\,|\, into)$... that is the possibility of context words given a center word. 
___
<img src="https://drive.google.com/uc?export=view&id=1Zxso4cOoW0lE2D8r5Zgkqleq05FUVcwk" width="800"/>
___
Then we move forward to the next word "banking" and we repeat the calculation on the probabilities. 


Next we need to find the objective function for each word position $t=1, ..., T$ so that we can predict the possibility of context words within a window of fixed size $m$, given center word $w_{j}$.

The likelihood function is specified as follows: 
\begin{equation}
L(\theta)=\prod_{t=1}^{T}\prod_{\substack{-m\leq j \leq m\\ j\neq 0}}P(w_{t+j}|w_{t}; \theta)
\end{equation}
where $j$ is the size of the window; $\theta$ is all variables to be optimized

Our goal is to maximize the likelihood function and find the center word (or the context word) that comes with the highest probability, which is also the most similar word. 

We use a mathmatical trick: logic monotonic transformation to realize our goal, that is we minimize the negative log of the likelihood fuction and tranform it into a loss/cost function: 

\begin{equation}
J(\theta)=-\frac{1}{T}logL(\theta)=-\frac{1}{T}\sum_{t=1}^{T}\sum_{\substack{-m\leq j \leq m\\ j\neq 0}}log\,P(w_{t+j}|w_{t}; \theta)
\end{equation}

We want to minimize the objective function:
\begin{equation}
J(\theta)=-\frac{1}{T}\sum_{t=1}^{T}\sum_{\substack{-m\leq j \leq m\\ j\neq 0}}log\,P(w_{t+j}|w_{t}; \theta)
\end{equation}

But now the question is <font color='blue'>how to calculate $P(w_{t+j}|w_{t}; \theta)$</font>

Answer: We will use two vectors per word $w$:
- $v_{w}$ when w is a center word
- $u_{w}$ when w is a context word

Then for a center word *c* and a context word *o*:

\begin{equation}
P(o\,|\,c)=\frac{exp(u_{o}^{T}v_{c})}{\sum_{w \in V}exp(u_{w}^{T}v_{c})}
\end{equation}
___
<img src="https://drive.google.com/uc?export=view&id=1Ex1fQTC86_MZozWqIlLFmZ037LtIR3dX" width="800"/>
___



Now let analyze the possibility funtion and undertand the format of it. 

- The dot product in the numerator represent the words similarity, but [why?](https://en.wikipedia.org/wiki/Cosine_similarity). 
 - Here the concept of **Cosine Similarity** is used: The cosine of 0° is 1, and it is less than 1 for any angle in the interval (0,π] radians. It is thus a judgment of orientation and not magnitude: two vectors with the same orientation have a cosine similarity of 1, two vectors oriented at 90° relative to each other have a similarity of 0, and two vectors diametrically opposed have a similarity of -1, independent of their magnitude. The cosine similarity is particularly used in positive space, where the outcome is neatly bounded in  [0, 1].
  - Another simple way to undertand this is to consider 2 words as 2 numbers. The closer the value of the numbers , the larger is their product and thus the more similar they are. For example, $w_{1}=3$ & $w_{2}=3$, then $3\cdot 3=9$, two vectors are exact the same. If they ar every different $w_{1}=1$ & $w_{2}=3$, then $1\cdot 3=3$. 
___
<img src="https://drive.google.com/uc?export=view&id=1n9Xuvn5mDsy7EA3agiYqvcsK6m9cCCTu" width="800"/>
___

This is an example of **softmax function** $\mathbb{R}^{n} \rightarrow \mathbb{R}^{n}$ mentioned in **CS231n** and frequently used in Deep Learning: 

\begin{equation}
softmax(x_{i})=\frac{exp(x_{i})}{\sum_{j=1}^{n}exp(x_{j})}=p_{i}
\end{equation}

The softmax function maps arbitrary values $x_{i}$ to a probability distribution $p_{i}$
- <font color='blue'>“max”</font> because amplifies probability of largest $x_{i}$
- <font color='blue'>“soft” </font>because still assigns some probability to smaller $x_{i}$

Now what is paramter vector $\theta$ ?

$\theta$ represents all model paramters, in one long vector that has *d*-dimensional vectors and *V*-many words:

\begin{equation}
\theta=\begin{bmatrix}
    v_{aardvark} \\  v_{a} \\ \vdots \\ v_{zebra}      \\     
    v_{aardvark} \\  v_{a} \\ \vdots \\ v_{zebra}
\end{bmatrix}\in \mathbb{R}^{2dV}
\end{equation}

- Each component in the big vector $\theta$ is itself a vector with *d*-dimension(lenth of the context?). 
- Everyword has 2 vectors: For example, we will have a vector for aardvark as a center word with *d* dimension : $v_{aardvark}$ on the top, then we will have a vector for aardvark as a context word with *d* dimension : $v_{aardvark}$ in the middle. 
 -  using our previous example: $v_{into}$ on the top could be [problems, turning,  ** <font color='red'>into</font>**, banking, cirses] with into as the center word. $v_{into}$ in the middle could be [problems, turning,  into, ** <font color='red'>banking</font>**, cirses] with into as the context word.
 
- So in totally we will have $2dV$ number of components in this big vector $\theta$




So how do we get the optimal $\theta$? we actually use MLE to solve for this problem mathematically.


\begin{equation}
Max \,J(\theta)=\prod_{t=1}^{T}\prod_{\substack{-m\leq j \leq m\\ j\neq 0}}P(w_{t+j}|w_{t}; \theta)
\end{equation}

where \begin{equation}
J(\theta)=-\frac{1}{T}logL(\theta)=-\frac{1}{T}\sum_{t=1}^{T}\sum_{\substack{-m\leq j \leq m\\ j\neq 0}}log\,P(w_{t+j}|w_{t}; \theta)
\end{equation}
 and \begin{equation}
P(o\,|\,c)=\frac{exp(u_{o}^{T}v_{c})}{\sum_{w \in V}exp(u_{w}^{T}v_{c})}
\end{equation}

Since now we look at the problem of finding both the center word and the context word that minimize the loss function. We first take the differential of the loss function w.r.t $v_{c}$ then w.r.t $u_{o}$: 

\begin{equation}
\frac{\partial}{\partial v_{c}}\, log\frac{exp(u_{o}^{T}v_{c})}{\sum_{w \in V}exp(u_{w}^{T}v_{c})}\\
=\frac{\partial}{\partial v_{c}}\, log\, exp(u_{o}^{T}v_{c}) - \frac{\partial}{\partial v_{c}}log\sum_{w \in V}exp(u_{w}^{T}v_{c})\\
=u_{o}-\frac{\sum_{w'=1}^{V} exp(u_{w'}^{T}v_{c})\cdot u_{w'}}{\sum_{w=1}^{V} exp(u_{w}^{T}v_{c})}\\
=\sum_{w'=1}^{V} \Big(\underbrace{\frac{ exp(u_{w'}^{T}v_{c})}{\sum_{w=1}^{V} exp(u_{w}^{T}v_{c})}}_{P(w'|c)} \cdot u_{w'}\Big)\\
=\underbrace{u_{0}}_{Observed \, representation \\of\, the\, context\, word }-\underbrace{\sum_{w'=1}^{V} P(w'|c)\cdot u_{w'}}_{Expected \, representation \\of\, the\, context\, word\, from \\the\, model}
\end{equation}


How does $u_{o}^{T}v_{c}$ looks like in matrix form? Consider the outside context matrix and center matrix as follows:

\begin{equation}
U\,(outside)=\begin{bmatrix}
   . . . . . \\  . . . . .  \\ . . . . . \\ . . . . .   \\     
   . . . . .  \\ . . . . . 
\end{bmatrix}\quad
V\,(center)=\begin{bmatrix}
   . . . . . \\  . . . . .  \\ . . . . . \\ \mathbf{\color{red}{. . . . .  }} \\     
   . . . . .  \\ . . . . . 
\end{bmatrix}
\end{equation}

First, you may notice that the words are all expressed in rows. It turns out that in many deep learninf packages,  such as TensorFlow, Pytorch, etc., for their word vectors, the word vectors are represented as rows. 

Now suppose we only look at the 4th center word vector, then we multiply the context word matrix $U$ with this vector, then we get:

\begin{equation}
U\, \cdot v_{4}^{T} =\begin{bmatrix}
   . \\  .  \\ .  \\ .   \\     
   .   \\ .  
\end{bmatrix}\quad
softmax(U\, \cdot v_{4}^{T}) =\begin{bmatrix}
   . \\  .  \\ .  \\ .   \\     
   .   \\ .  
\end{bmatrix}
\end{equation}

#### 4.2  Word2vec Optimization: Gradient Descent

Gradient Descent is an algorithm to minimize $J(\theta)$. The idea is for current value of $\theta$, calculate gradient of $J(\theta)$ , then take small step in direction of negative gradient. Repeat.

The update quation in matrix notation is: 
\begin{equation}
\theta^{new}=\theta^{old}-\alpha \Delta_{\theta}J(\theta)
\end{equation}

where $\alpha$ is the step size or the learning rate. 

The update equation for single parameter is:

\begin{equation}
\theta_{j}^{new}=\theta_{j}^{old}-\alpha \frac{\partial}{\partial \theta_{j}^{old}}J(\theta)
\end{equation}

The algorithm is:

In [0]:
while True:
  theta_grad = evaluate_gradient(J, corpus, theta)
  theta = theta col- alpha * theta_grad

The example application of this word2vec model can be found [HERE Gensim word vector visualization of various word vectors](https://colab.research.google.com/drive/1qp6_wi7c2nwkQv07Mf_CEkhFv7bc_QS5#scrollTo=TjAoHofQX80s).